<a href="https://colab.research.google.com/github/harini-si/saidl-assignment/blob/main/SSL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import numpy as np
import pandas as pd
import shutil, time, os, requests, random, copy

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms, models
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import PIL
from PIL import Image
from torchvision.models import resnet50
from torchvision.models import resnet18
import torch.nn.functional as F

In [2]:
device='cuda'

In [3]:

feature_transform = transforms.Compose([
        transforms.RandomResizedCrop(96),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomApply([transforms.ColorJitter(0.4, 0.4, 0.4, 0.1)], p=0.8),
        transforms.RandomGrayscale(p=0.2),
        transforms.ToTensor(),
        transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])
    ])
transform=transforms.Compose([transforms.RandomResizedCrop(96),transforms.ToTensor()])

In [4]:
class data(datasets.STL10):

    def __getitem__(self, index):
       
       
        img1 = self.data[index]

        img = Image.fromarray(np.transpose(img1, (1, 2, 0)))

        if self.transform is not None:
            img1 = self.transform(img)
            img2 = self.transform(img)
        else:
            img2 = img1 = img


        return img1, img2


In [5]:
unlabeled_data = data(root="./data", split='unlabeled', download=True, transform=feature_transform)
train_data=datasets.STL10(root="./data",split='train',download=True,transform=transform)
test_data=datasets.STL10(root="./data",split='test',download=True,transform=ToTensor())
unlabeled1=torch.utils.data.Subset(unlabeled_data,range(0,int (len(unlabeled_data)/2), 2))
unlabeled_loader=DataLoader(unlabeled1,shuffle=True,batch_size=128)
train_loader=DataLoader(train_data,shuffle=True,batch_size=64)
test_loader=DataLoader(test_data,shuffle=True,batch_size=64)

  0%|          | 0/2640397119 [00:00<?, ?it/s]

Extracting ./data/stl10_binary.tar.gz to ./data
Files already downloaded and verified
Files already downloaded and verified


In [6]:
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
    def forward(self, x):
        return x

In [7]:
class SimCLR(nn.Module):
    def __init__(self):
        super(SimCLR,self).__init__()
        self.pretrained = models.resnet18(pretrained=True)
        
        self.pretrained.conv1 = nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
        self.pretrained.maxpool = Identity()
        
        self.pretrained.fc = Identity()
        
        for p in self.pretrained.parameters():
            p.requires_grad = True
       
        self.g = nn.Sequential(nn.Linear(512, 512, bias=False), nn.BatchNorm1d(512),
                               nn.ReLU(inplace=True), nn.Linear(512,128, bias=True))
        
    def forward(self, x):
        x = self.pretrained(x)
        feature = torch.flatten(x, start_dim=1)
        out = self.g(feature)
        return F.normalize(out, dim=-1)

In [ ]:
simclr_model.pretrained

In [12]:

class contrastive_loss(nn.Module):
    def __init__(self, normalize=False):
        super(contrastive_loss, self).__init__()
   
        self.normalize = normalize

    def forward(self, zi, zj):

        x = torch.cat((zi, zj), dim=0)

        mat = torch.mm(x, x.T)
        if self.normalize:
            a = torch.mm(torch.linalg.matrix_norm(x, dim=1).unsqueeze(1), torch.linalg.matrix_norm(x, dim=1).unsqueeze(1).T)
            mat = mat /( a+1e-16)

        similarity = torch.exp(mat / 0.2)

        if self.normalize:
            a = torch.torch.linalg.matrix_norm(zi, dim=1) * torch.torch.linalg.matrix_norm(zj, dim=1)
            sim = torch.exp(torch.sum(zi * zj, dim=-1) / (a / 0.2))
        else:
            sim = torch.exp(torch.sum(zi * zj, dim=-1) / 0.2)

        sim = torch.cat((sim, sim), dim=0)

        normalize = torch.exp(torch.ones(x.size(0)) / 0.2)
       
        loss = torch.mean(-torch.log(sim / (torch.sum(similarity, dim=-1) - normalize)))

        return loss

In [9]:
simclr_model = SimCLR().to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(simclr_model.parameters(),lr=0.01)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [ ]:
simclr_model

In [13]:
def train(model, loader, epoch, num_epochs):

    loss_func = contrastive_loss()


    model.train()
  
    total_loss = 0.0
    total_steps=len(loader)

    for i, (x_i, x_j) in enumerate(loader):

        x_i = x_i.to(device) 
        x_j = x_j.to(device) 

        z_i = simclr_model(x_i)
        z_j = simclr_model(x_j)

        loss = loss_func(z_i, z_j)
        loss.backward()

        optimizer.step()
        simclr_model.zero_grad()

        total_loss += loss.item()
        if i%100==0:
          print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{total_steps}], Loss: {loss.item():.4f}')



In [ ]:
num_epochs=8
for epoch in range(num_epochs):
  train(simclr_model, unlabeled_loader,epoch, num_epochs)


In [12]:
class LinearClassifier(nn.Module):
    def __init__(self,model, input_dim=512, num_classes=10):
        super(LinearClassifier, self).__init__()
        self.input_dim = input_dim
        self.num_classes = num_classes
        self.encoder= model

        for p in self.encoder.parameters():
            p.requires_grad = False
        self.fc = nn.Linear(self.input_dim, self.num_classes)

    def forward(self, x):
        out = self.encoder.pretrained(x)
        out = self.fc(out)
        return out

In [13]:
classifier_model= LinearClassifier(simclr_model).to(device)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(classifier_model.parameters(),lr=0.001)
num_epochs1=15

In [14]:
#def test loop and supervised training
def test_loop(dataloader, model):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    model.eval()
    with torch.no_grad():
        for X, y in dataloader:
            X=X.to('cuda')
            y=y.to('cuda')
            pred = model(X)
            test_loss += loss_func(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    Accuracy=100*correct
    return Accuracy
n_total_steps = len(train_loader)
for epoch in range(num_epochs1):
   
    correct=0
    total=0
    for i, (images, labels) in enumerate(train_loader):
      
        images = images.to('cuda')
        labels = labels.to('cuda')

        
        outputs = classifier_model(images)
        loss = loss_func(outputs, labels)

        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
       
        
    
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
      

        if (i+1) % 20 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs1}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')
      
    
    accu= 100 * correct / len(train_data)
    test_accu=test_loop(test_loader,classifier_model)
    print(f'Epoch [{epoch+1}/{num_epochs1}], train_accuracy: {accu:.4f},test_accuracy: {test_accu:.4f} %')   


Epoch [1/20], Step [20/79], Loss: 1.5992
Epoch [1/20], Step [40/79], Loss: 1.8067
Epoch [1/20], Step [60/79], Loss: 1.5457
Epoch [1/20], train_accuracy: 32.8200,test_accuracy: 36.3500 %
Epoch [2/20], Step [20/79], Loss: 1.4903
Epoch [2/20], Step [40/79], Loss: 1.6157
Epoch [2/20], Step [60/79], Loss: 1.4983
Epoch [2/20], train_accuracy: 37.4800,test_accuracy: 39.2875 %
Epoch [3/20], Step [20/79], Loss: 1.5767
Epoch [3/20], Step [40/79], Loss: 1.6289
Epoch [3/20], Step [60/79], Loss: 1.4111
Epoch [3/20], train_accuracy: 38.8600,test_accuracy: 40.9125 %
Epoch [4/20], Step [20/79], Loss: 1.6718
Epoch [4/20], Step [40/79], Loss: 1.4896
Epoch [4/20], Step [60/79], Loss: 1.5708
Epoch [4/20], train_accuracy: 39.9400,test_accuracy: 40.1125 %
Epoch [5/20], Step [20/79], Loss: 1.4652
Epoch [5/20], Step [40/79], Loss: 1.9431
Epoch [5/20], Step [60/79], Loss: 1.5832
Epoch [5/20], train_accuracy: 40.0200,test_accuracy: 40.5625 %
Epoch [6/20], Step [20/79], Loss: 1.6476
Epoch [6/20], Step [40/79], L